In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

# change path if necessary
import sys
my_path = r'D:\Documents\etudes\epfl\MA1\cours\MachineLearning\Project2'
sys.path.insert(0,my_path + r'/code/COMMON')

# imports
import numpy as np 
import matplotlib.pyplot as plt

In [2]:
from scipy.signal import convolve

def process_tweet(X_tweet, method):
    
    # mean of the features
    if method is 'mean':
        X_tweet_processed = np.mean(X_tweet, 0)
    
    return X_tweet_processed

# Load Glove model

In [3]:
from glove import Corpus
from glove import Glove

In [4]:
import os
os.chdir(r'D:/Documents/etudes/epfl/MA1/cours/MachineLearning/Project2/data/twitter_datasets_stanford/')

In [5]:
Glove = Glove.load_stanford('glove.twitter.27B.25d.txt')

# Load Stop words

In [6]:
import os
os.chdir(r'D:/Documents/etudes/epfl/MA1/cours/MachineLearning/Project2/data/stop_words/')

In [7]:
filename_stopwords = 'stop_word_freq_min_100_ratio_marg_0.2.txt'

stop_words = []
with open(filename_stopwords, 'r', encoding='utf-8-sig') as f:
    for line in f:
        stop_words.append(line.lstrip().split()[0])
    del stop_words[-1]
    
print("File :", filename_stopwords)
print("Number of stop words :", len(stop_words))

File : stop_word_freq_min_100_ratio_marg_0.2.txt
Number of stop words : 1206


In [8]:
stop_words

["'",
 '&',
 '*',
 ',',
 ';',
 '\\',
 '_',
 '__',
 '`',
 '{',
 '}',
 '~',
 '<elong>',
 '<number>d',
 '<number>day',
 '<number>days',
 '<number>ish',
 '<number>th',
 '<number>years',
 '<number>yrs',
 '=',
 '->',
 'a',
 'aaaw',
 'aah',
 'aaron',
 'abby',
 'above',
 'abs',
 'abt',
 'account',
 'accounts',
 'achieve',
 'act',
 'actor',
 'actually',
 'adam',
 'add',
 'address',
 'adele',
 'advance',
 'after',
 'again',
 'ah',
 'ahh',
 'aim',
 'aint',
 'aj',
 'alcohol',
 'alex',
 'algebra',
 'alive',
 'all',
 'allah',
 'almost',
 'already',
 'although',
 'am',
 'amanda',
 'amsterdam',
 'and',
 'andy',
 'angel',
 'anger',
 'anniversary',
 'announce',
 'announced',
 'annoy',
 'anonymous',
 'another',
 'answer',
 'answered',
 'any',
 'anyone',
 'anything',
 'apa',
 'apologies',
 'apologize',
 'app',
 'appearance',
 'apply',
 'apprentice',
 'april',
 'arena',
 'argentina',
 'arguing',
 'argument',
 'arrive',
 'arse',
 'as',
 'asap',
 'ashley',
 'asked',
 'asks',
 'asshole',
 'at',
 'atl',
 'aunt

In [9]:
print("happy" in stop_words)
print("another" in stop_words)

False
True


# Build tweet vectors

In [10]:
import os
os.chdir(r'D:\Documents\etudes\epfl\MA1\cours\MachineLearning\Project2\data\twitter_datasets_epfl\short')

In [11]:
# method to build tweet vector
method = "mean"

In [12]:
# build positive tweet feature set
X_pos = []

with open('train_pos_processed.txt') as f:
    for line in f:
        tweet = line.lstrip().split()
        X_tweet = []
        for word in tweet:
            if word in Glove.dictionary and not word in stop_words:
                word_vector = Glove.word_vectors[Glove.dictionary.get(word)]
                X_tweet.append(word_vector)
                
        if X_tweet:
            X_tweet_processed = process_tweet(X_tweet, method)
            X_pos.append(X_tweet_processed)
        
X_pos = np.array(X_pos)

In [13]:
# build negative tweet feature set
X_neg = []

with open('train_neg_processed.txt') as f:
    for line in f:
        tweet = line.lstrip().split()
        X_tweet = []
        for word in tweet:
            if word in Glove.dictionary and not word in stop_words:
                word_vector = Glove.word_vectors[Glove.dictionary.get(word)]
                X_tweet.append(word_vector)
                
        if X_tweet:
            X_tweet_processed = process_tweet(X_tweet, method)
            X_neg.append(X_tweet_processed)
        
X_neg = np.array(X_neg) 

In [14]:
y_pos = np.ones(X_pos.shape[0])
y_neg = -np.ones(X_neg.shape[0])

In [15]:
N_samples_train = -1 

# cut samples
X_pos_cut = X_pos[:N_samples_train,:]
X_neg_cut = X_neg[:N_samples_train,:]

# cut targets
y_pos_cut = y_pos[:N_samples_train]
y_neg_cut = y_neg[:N_samples_train]

In [16]:
X_pos_neg = np.concatenate([X_pos_cut, X_neg_cut])
y_pos_neg = np.concatenate([y_pos_cut, y_neg_cut])

# Standardization

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_pos_neg)
X_pos_neg_scaled = scaler.transform(X_pos_neg)

# Train SVM 

### C-Support Vector Classification 

In [ ]:
from sklearn.svm import SVC

clf = SVC()
clf.fit(X_pos_neg_scaled, y_pos_neg) 

### Nu-Support Vector Classification.

In [ ]:
from sklearn.svm import NuSVC

clf = NuSVC()
clf.fit(X_pos_neg_scaled, y_pos_neg) 

### Linear Support Vector Classification

In [ ]:
from sklearn.svm import LinearSVC

clf = LinearSVC(penalty='l1', dual = False)
clf.fit(X_pos_neg_scaled, y_pos_neg) 

# Predict

In [ ]:
import os
os.chdir(r'D:\Documents\etudes\epfl\MA1\cours\MachineLearning\Project2\data\twitter_datasets_epfl\short')

In [ ]:
X_test = []

with open('test_data_no_id_processed.txt') as f:
    
    for line in f:
        
        tweet = line.lstrip().split()
        X_tweet = []
        
        for word in tweet:
            
            if word in Glove.dictionary and not word in stop_words:
                
                word_vector = Glove.word_vectors[Glove.dictionary.get(word)]
                X_tweet.append(word_vector)
                
        if X_tweet:
            
            X_tweet_processed = process_tweet(X_tweet, method)
            X_test.append(X_tweet_processed)
        
X_test = np.array(X_test) 
print(X_test.shape)

In [ ]:
# standarization
X_test_scaled = scaler.transform(X_test)

In [ ]:
# predict
y_pred = clf.predict(X_test)

# Submission

In [ ]:
import os
os.chdir(r'D:/Documents/etudes/epfl/MA1/cours/MachineLearning/Project2/data/submissions/')

In [ ]:
from create_csv_submission import create_csv_submission
import time
import datetime
i = datetime.datetime.now()

name = "sub_" + time.strftime("%d_%m_%Y") +  "_%sh_%smin" % (i.hour, i.minute)
#name = "vrf"
ids_test = range(1, X_test.shape[0]+1)
create_csv_submission(ids_test, y_pred, name)